In [1]:
import torch
from torch import nn
import pandas as pd

import math
import matplotlib.pyplot as plt
torch.manual_seed(111)

from sklearn.utils import shuffle
import numpy as np

import torchvision
import torchvision.transforms as transforms

device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
train_data_length = 1024

train_data = torch.randn((train_data_length, 27))

train_data[:, 0] = 2 * math.pi * torch.rand(train_data_length)

train_data[:, 1] = torch.sin(train_data[:, 0])

train_labels = torch.zeros(train_data_length)

train_set = [

    (train_data[i], train_labels[i]) for i in range(train_data_length)

]

print(train_set)
plt.plot(train_data[:, 0], train_data[:, 1], ".")

In [63]:
dataset = pd.read_csv('datasets/Fs_B_DANCE_WALK_0.5sec.csv')
dataset = dataset.drop(columns=['EMOTION_P', 'EMOTION_A', 'EMOTION_D'])
#train_data = dataset.sample(frac=1, random_state=42)
train_data = dataset.iloc[:18400]

print("No Training Samples:",train_data.shape[0])

train_data_length = train_data.shape[0]

train_data = torch.tensor(train_data.values, dtype=torch.float32)

train_labels = torch.zeros(train_data_length)

train_set = [

    (train_data[i],  train_labels[i]) for i in range(train_data_length)

]

#print(train_set)

No Training Samples: 18400


In [64]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

In [65]:
class Discriminator(nn.Module):

    def __init__(self):

        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(27, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1),
            nn.Sigmoid(),

        )


    def forward(self, x):

        output = self.model(x)

        return output


In [66]:
discriminator = Discriminator()

In [67]:
class Generator(nn.Module):

    def __init__(self):

        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(5, 12),
            nn.ReLU(),
            
            nn.Linear(12, 27),
        )


    def forward(self, x):
        output = self.model(x)

        return output


generator = Generator()

In [68]:
lr = 0.001

num_epochs = 300

loss_function = nn.MSELoss()

In [69]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)

optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [70]:
for epoch in range(num_epochs):

    for n, (real_samples, _) in enumerate(train_loader):

        # Data for training the discriminator

        real_samples_labels = torch.ones((batch_size, 1))

        latent_space_samples = torch.randn((batch_size, 5))

        generated_samples = generator(latent_space_samples)

        generated_samples_labels = torch.zeros((batch_size, 1))

        all_samples = torch.cat((real_samples, generated_samples))

        all_samples_labels = torch.cat(

            (real_samples_labels, generated_samples_labels)

        )
        

        # Training the discriminator

        discriminator.zero_grad()

        output_discriminator = discriminator(all_samples)

        loss_discriminator = loss_function(

            output_discriminator, all_samples_labels)

        loss_discriminator.backward()

        optimizer_discriminator.step()


        # Data for training the generator

        latent_space_samples = torch.randn((batch_size, 5))


        # Training the generator

        generator.zero_grad()

        generated_samples = generator(latent_space_samples)

        output_discriminator_generated = discriminator(generated_samples)

        loss_generator = loss_function(

            output_discriminator_generated, real_samples_labels

        )

        loss_generator.backward()

        optimizer_generator.step()


        # Show loss

        if epoch % 10 == 0 and n == batch_size - 1:

            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")

            print(f"Epoch: {epoch} Loss G.: {loss_generator}")

Epoch: 0 Loss D.: 0.01353990938514471
Epoch: 0 Loss G.: 0.8258992433547974
Epoch: 10 Loss D.: 0.14478416740894318
Epoch: 10 Loss G.: 0.6103126406669617
Epoch: 20 Loss D.: 0.2086859792470932
Epoch: 20 Loss G.: 0.6199708580970764
Epoch: 30 Loss D.: 0.11921965330839157
Epoch: 30 Loss G.: 0.5277076363563538
Epoch: 40 Loss D.: 0.08849553763866425
Epoch: 40 Loss G.: 0.6822596788406372
Epoch: 50 Loss D.: 0.12051056325435638
Epoch: 50 Loss G.: 0.6714287996292114
Epoch: 60 Loss D.: 0.1403384655714035
Epoch: 60 Loss G.: 0.8170068860054016
Epoch: 70 Loss D.: 0.1081857979297638
Epoch: 70 Loss G.: 0.6716679334640503
Epoch: 80 Loss D.: 0.12228119373321533
Epoch: 80 Loss G.: 0.6698785424232483
Epoch: 90 Loss D.: 0.09698949754238129
Epoch: 90 Loss G.: 0.6768026947975159
Epoch: 100 Loss D.: 0.08176574856042862
Epoch: 100 Loss G.: 0.6604031920433044
Epoch: 110 Loss D.: 0.1335138976573944
Epoch: 110 Loss G.: 0.6915494203567505
Epoch: 120 Loss D.: 0.07217991352081299
Epoch: 120 Loss G.: 0.7187076807022095

In [ ]:
latent_space_samples = torch.randn(10, 27)
generated_samples = generator(latent_space_samples)

In [71]:
generated_samples = generated_samples.detach()
print(generated_samples[0])


tensor([ 0.7151,  0.3648,  0.3633,  0.6360,  0.4562,  0.2526,  0.3801,  0.3801,
         0.3294,  0.3166, -0.0127,  0.0372,  0.2131,  1.0413,  0.4555,  0.0682,
         0.1086,  1.4817,  0.4525,  0.4389,  0.6356,  0.3728,  4.8970,  1.1899,
         0.1314,  2.8897,  0.9827])
